Import Libraries
TO DO
1) Add larger test dataset
2) Add Speed Tests
3) Create Real Polygons for Testing Purpose

In [1]:
import numpy as np
from osgeo import gdal
from scipy import stats
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
import math
from math import floor
 

In [2]:
## Warning You will need 65 gb to create the memory map for the CU imper sum down data type

In [3]:
Grid_Name  = "data\\nlcd_2011_impervious_2011_edition_2014_10_10\\nlcd_2011_impervious_2011_edition_2014_10_10.img"
memory_map_Name = "data\\nlcd_imp.mmp"

r = gdal.Open(Grid_Name)
r.GetGeoTransform()
r.RasterXSize
band = r.GetRasterBand(1)
print(band.GetBlockSize())

DataType = band.DataType
DataType = gdal.GetDataTypeName(DataType)
print(DataType)



[64, 64]
Byte


In [4]:
## calculate Values of the raster
block_size = 100;
ysize_mmp = math.ceil(r.RasterYSize / block_size)*block_size + 0; 
xsize_mmp = math.ceil(r.RasterXSize / block_size)*block_size + 0;
rnsze = (ysize_mmp) * (xsize_mmp);

data = np.memmap(memory_map_Name, dtype='uint8', mode='w+', shape=(ysize_mmp * xsize_mmp))
##data = np.memmap(memory_map_Name, dtype='float32', mode='w+', shape=(ysize_mmp * xsize_mmp))
 

In [5]:
## rmp function to turn row and column major
## into blocks
## Yt = RasterYSize, Xt = RasterXSize

def rmp(Yt,Xt,Y,X,blcksze):
    ##Y += 1; X += 1;
    
    DY = (floor((Y)/blcksze)) * blcksze
 
    dy = Y - DY
 
    DX = (floor((X)/blcksze)) * blcksze

    
    dx = X - DX;

    
    p1 = DX * Yt;
    p2 = DY * blcksze;
    p3 = blcksze * dx ;
    p4 = dy   
    pos = p1 + p2 + p3 + p4;
    return(int(pos));


In [6]:
rmp(100,100,9,9,10)


99

In [26]:
# Function to read the raster as arrays for the chosen block size.
def create_cp_grid(blcksze, raster, mmp, ysize_mmp, xsize_mmp):
    ds = gdal.Open(raster)
    band = ds.GetRasterBand(1)

    blocks = 0
    ##for y in range(0, 1000, blcksze):
    for y in range(1, band.YSize, blcksze):
        print(y)
        print(y/band.YSize)
        if y + blcksze < band.YSize:
            rows = blcksze
        else:
            rows = band.YSize - y
        ##for x in range(0, 1000, blcksze):
        for x in range(0, band.XSize, blcksze):
            if x + blcksze < band.XSize:
                cols = blcksze
            else:
                cols = band.XSize - x
            ##array = band.ReadAsArray(x, y, cols, rows)
            i = list([]);
            for ywi in range(0,rows):
                for xwi in range(0,cols):
                      i.append(rmp(ysize_mmp,xsize_mmp,ywi+0+y,xwi+0+x,blcksze))
            mmp[i] = band.ReadAsArray(x, y, cols, rows).flat
            ##print(i[0])
            del i
            blocks += 1
    band = None
    ds = None


In [28]:

create_cp_grid(100, Grid_Name, data, ysize_mmp, xsize_mmp)

0
0.0
100
0.0009576342603232973
200
0.0019152685206465945
300
0.002872902780969892
400
0.003830537041293189
500
0.004788171301616487
600
0.005745805561939784
700
0.006703439822263082
800
0.007661074082586378
900
0.008618708342909676


In [31]:
def tst(y,x):
    print( "orig is " + str(band.ReadAsArray(x, y, 1, 1)[0]))
    ##data[1]
    print( "new is " + str(data[rmp(ysize_mmp,xsize_mmp,y,x,100)]))



In [34]:
tst(0,0)
tst(99,99)
tst(100,100)
##tst(11290,16899)
rmp(ysize_mmp,xsize_mmp,1,1,100)
##rmp(ysize_mmp,xsize_mmp,100,100,100)

orig is [127]
new is 127
orig is [127]
new is 127
orig is [127]
new is 127


101

In [18]:
data[rmp(ysize_mmp,xsize_mmp,10,10,100)]

127

In [88]:
band.ReadAsArray(r.RasterXSize-1, r.RasterYSize-1, 1, 1)[0]

array([127], dtype=uint8)